# 行为型-状态模式

## 模式说明

对象一个时间处于一个状态，对象会在这些状态中间切换。状态模式有一个上下文类以及不同的状态类，其核心就是将上下文和状态解耦。

## 实例

### 实例1

先来看一个最简单的例子：

In [28]:
class Computer:
    def __init__(self):
        self.state = ClosedState()

    def do_work(self):
        self.state.do_work()


class OpenedState:
    def do_work(self):
        print("Opened: do work!")


class ClosedState:
    def do_work(self):
        print("Closed: do work!")


class ReadState:
    def do_work(self):
        print("Reading: do work!")

In [29]:
computer = Computer()
computer.do_work()
computer.state = OpenedState()
computer.do_work()
computer.state = ReadState()
computer.do_work()

Closed: do work!
Opened: do work!
Reading: do work!


上面的代码将状态和上下文进行了解耦，省略了大量的`if...else`，如果不使用状态模式，那么会是这样：

In [32]:
class Computer:
    def __init__(self):
        self.state = "close"

    def do_work(self):
        if self.state == "close":
            print("Closed: do work!")
        elif self.state == "open":
            print("Opend: do work!")
        else:
            print("Reading: do work!")

In [33]:
computer = Computer()
computer.do_work()
computer.state = "open"
computer.do_work()
computer.state = "read"
computer.do_work()

Closed: do work!
Opend: do work!
Reading: do work!


上面的代码中，切换由上下文完成，状态之间的切换没有限制，比如，可以直接从`close`状态切换到`read`状态。如果状态切换存在限制，则上下文中又增加一大堆`if..else`的判断语句。此时可以进一步优化代码，将切换也放到状态类中，每一个状态类只需要管理好自己状态下的切换限制：

In [47]:
class CloseState:
    @staticmethod
    def switch(computer, state):
        if state != OpenState: # close状态下，只能切换到open状态
            raise RuntimeError("can only switch to open state")
        computer.state = state
    
    @staticmethod
    def do_work():
        print("close: do work!")
        

class OpenState:
    @staticmethod
    def switch(computer, state):
        if state == OpenState:  # open状态下，可以切换到close或者read，但是不能重复open
            raise RuntimeError("already open")
        computer.state = state
    
    @staticmethod
    def do_work():
        print("Open: do work!")
        
        
class Computer:
    def __init__(self):
        self.state = CloseState
    
    def switch(self, state):
        self.state.switch(self, state)
        
    def do_work(self):
        self.state.do_work()

In [49]:
computer = Computer()
computer.do_work()
try:
    computer.switch(CloseState)
except RuntimeError as e:
    print(e)
computer.switch(OpenState)
computer.do_work()

close: do work!
can only switch to open state
Open: do work!


上面的代码，虽然把状态与上下文进行了比较彻底的解耦。但是在状态类的切换函数中，仍然存在很多`if..else`判断语句。《python设计模式第二版》提供了一个比较巧妙的方法：

In [62]:
class State:
    allowed = []

    def switch(self, state):
        if state.name not in self.allowed:
            raise RuntimeError(f"Can't swith from {self.name} to {state.name}")
        """
        注意这里，这种方法并没有像上面的代码一样，将computer实例传递给switch，显式的设置computer的当前状态，而是直接替换掉状态实例的所属类
        所以，computer的状态始终是初始化时设置的实例，状态切换，切换的是这个实例所属的类。
        由于没有将上下文传递给switch方法，因此无法使用上下文的数据，可以再做修改，将上下文实例作为参数传递给switch
        """
        self.__class__ = state
        # 如果上下文实例作为参数传递进来，则可以不用上面的代码，直接设置上下文的state属性
        # computer.state = state  # 此时的state为状态类的实例，而不是状态类


class OpenState(State):
    name = 'open'
    allowed = ['close']

    @staticmethod
    def do_work():
        print("Open: do work!")


class CloseState(State):
    name = 'close'
    allowed = ['open']

    @staticmethod
    def do_work():
        print("Close: do work!")


class Computer:
    def __init__(self):
        self.state = CloseState()
        
    def switch(self, state):
        self.state.switch(state)
        
    def do_work(self):
        self.state.do_work()
        
        
computer = Computer()
computer.do_work()
computer.switch(OpenState)
computer.do_work()
try:
    computer.switch(OpenState)
except RuntimeError as e:
    print(e)

Close: do work!
Open: do work!
Can't swith from open to open


如果状态不是很多的话，还可以借鉴《python cookbook第二版》8.19的方法，不使用`switch`，而是直接把所有状态都设置为上下文的方法，状态切换直接调用相应的方法就好了：

In [54]:
class CloseState:
    @staticmethod
    def close(computer):
        raise RuntimeError("Closed already!")
    
    @staticmethod
    def open(computer):
        computer.state = OpenState
          

class OpenState:
    @staticmethod
    def open(computer):
        raise RuntimeError("Opened already!")
    
    @staticmethod
    def close(computer):
        computer.state = CloseState
    
    @staticmethod
    def do_work():
        print("open: do work!")
        

class Computer:
    def __init__(self):
        self.state = CloseState
    
    def close(self):
        self.state.close(self)
        
    def open(self):
        self.state.open(self)
    
    def do_work(self):
        self.state.do_work()
        

computer = Computer()
try:
    computer.close()
except RuntimeError as e:
    print(e)
computer.open()
computer.do_work()

Closed already!
open: do work!


### 实例2

来看一个《python3面对对象编程》第8章的例子。这个例子使用状态模式读取xml文件，是一个实际生产中的例子，有如下的xml文件：
```xml
<book>
    <author>Dusty Phillips</author>
    <publisher>Packet Publishing</publisher>
    <title>Python 3 Object Oriented Programming</title>
    <content>
        <chapter>
            <number>1</number>
            <title>Object Oriented Design</title>
        </chapter>
        <chapter>
            <number>2</number>
            <title>Objects In Python</title>
        </chapter>
    </content>
</book>
```
可以使用状态模式对其进行解析。

## 总结